In [1]:
import mysql.connector
import os
import pandas as pd

# Adding audit_report csv to the database

We have added the auditor report to the database and integrated it with the `visits` table. 
President Naledi appointed this auditor to investigate anomalous records identified in the `water_quality` table. This auditor re-visited a randomly selected subset of 1620 records, as provided by Chidi Kunto, with the aim of re-recording the quality_scores. The auditor also spoke to citizens close to the sources in order to understand why there were discrepancies.


Joining the audit report with visits table

In [9]:

# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# The query retrieves records by joining the auditor_report and visits tables on location_id, 
# returning the location_id (as both audit_location and visit_location), the true_water_source_score from auditor_report,
# and the record_id from visits.
query = """
    SELECT
        auditor_report.location_id AS audit_location,
        auditor_report.true_water_source_score,
        visits.location_id AS visit_location,
        visits.record_id
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id;
"""

# Execute the query
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchall()

# Fetch column headers
column_headers = [desc[0] for desc in mycursor.description]

# Create a pandas DataFrame
df = pd.DataFrame(results, columns=column_headers)

# Print the DataFrame
print(df)

# Close the cursor and connection
mycursor.close()
cnx.close()


     audit_location  true_water_source_score visit_location  record_id
0         SoRu34980                        1      SoRu34980       5185
1         AkRu08112                        3      AkRu08112      59367
2         AkLu02044                        0      AkLu02044      37379
3         AkHa00421                        3      AkHa00421      51627
4         SoRu35221                        0      SoRu35221      28758
...             ...                      ...            ...        ...
2693      SoRu36378                        9      SoRu36378      19733
2694      KiRu27180                        6      KiRu27180      42498
2695      HaRu16981                        9      HaRu16981       2716
2696      HaDe16326                        1      HaDe16326      41884
2697      HaRu20394                        9      HaRu20394      24662

[2698 rows x 4 columns]


Now that we have the record_id for each location, our next step is to retrieve the corresponding scores from the water_quality table. We
are particularly interested in the subjective_quality_score. To do this, we'll JOIN the visits table and the water_quality table, using the
record_id as the connecting key.

In [11]:
# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# Define the query
query = """
    SELECT
        auditor_report.location_id AS audit_location,
        auditor_report.true_water_source_score,
        visits.location_id AS visit_location,
        visits.record_id,
        water_quality.subjective_quality_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id;
"""

# Execute the query
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchall()

# Fetch column headers
column_headers = [desc[0] for desc in mycursor.description]

# Create a pandas DataFrame
df = pd.DataFrame(results, columns=column_headers)

# Print the DataFrame
print(df)

# Close the cursor and connection
mycursor.close()
cnx.close()


     audit_location  true_water_source_score visit_location  record_id  \
0         SoRu34980                        1      SoRu34980       5185   
1         AkRu08112                        3      AkRu08112      59367   
2         AkLu02044                        0      AkLu02044      37379   
3         AkHa00421                        3      AkHa00421      51627   
4         SoRu35221                        0      SoRu35221      28758   
...             ...                      ...            ...        ...   
2693      SoRu36378                        9      SoRu36378      19733   
2694      KiRu27180                        6      KiRu27180      42498   
2695      HaRu16981                        9      HaRu16981       2716   
2696      HaDe16326                        1      HaDe16326      41884   
2697      HaRu20394                        9      HaRu20394      24662   

      subjective_quality_score  
0                            1  
1                            3  
2           

It doesn't matter if  columns are in a different format, because we are about to clean this up a bit. Since it is a duplicate, we can drop one of
the location_id columns. Let's leave record_id and rename the scores to surveyor_score and auditor_score to make it clear which scores
we're looking at in the results set.

In [14]:
# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()
# Define the query
query = """
    SELECT
        auditor_report.location_id,
        visits.record_id,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id;
"""
# Execute the query
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchall()

# Fetch column headers
column_headers = [desc[0] for desc in mycursor.description]

# Create a pandas DataFrame
df = pd.DataFrame(results, columns=column_headers)

# Print the DataFrame
print(df)

# Close the cursor and connection
mycursor.close()
cnx.close()


     location_id  record_id  auditor_score  surveyor_score
0      SoRu34980       5185              1               1
1      AkRu08112      59367              3               3
2      AkLu02044      37379              0               0
3      AkHa00421      51627              3               3
4      SoRu35221      28758              0               0
...          ...        ...            ...             ...
2693   SoRu36378      19733              9               9
2694   KiRu27180      42498              6               6
2695   HaRu16981       2716              9               9
2696   HaDe16326      41884              1               1
2697   HaRu20394      24662              9               9

[2698 rows x 4 columns]


A good starting point is to check if the auditor's and exployees' scores agree. There are many ways to do it. We can have a
WHERE clause and check if surveyor_score = auditor_score

In [15]:
# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# Define the query to check if the scores agree
query = """
    SELECT
        visits.record_id,
        auditor_report.location_id AS audit_location,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    WHERE
        auditor_report.true_water_source_score = water_quality.subjective_quality_score;
"""

# Execute the query
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchall()

# Fetch column headers
column_headers = [desc[0] for desc in mycursor.description]

# Create a pandas DataFrame
df = pd.DataFrame(results, columns=column_headers)

# Print the DataFrame
print(df)

# Close the cursor and connection
mycursor.close()
cnx.close()



      record_id audit_location  auditor_score  surveyor_score
0          5185      SoRu34980              1               1
1         59367      AkRu08112              3               3
2         37379      AkLu02044              0               0
3         51627      AkHa00421              3               3
4         28758      SoRu35221              0               0
...         ...            ...            ...             ...
2500      19733      SoRu36378              9               9
2501      42498      KiRu27180              6               6
2502       2716      HaRu16981              9               9
2503      41884      HaDe16326              1               1
2504      24662      HaRu20394              9               9

[2505 rows x 4 columns]


We got 2505 rows, Some of the locations were visited multiple times, so these records are duplicated here. To fix it, we set visits.visit_count
= 1 in the WHERE clause. We make  sure we reference the alias we used for visits in the join.

In [16]:
# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# Define the query to check if the scores agree
query = """
    SELECT
        visits.record_id,
        auditor_report.location_id AS audit_location,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    WHERE
        auditor_report.true_water_source_score = water_quality.subjective_quality_score
        and visits.visit_count=1;
"""

# Execute the query
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchall()

# Fetch column headers
column_headers = [desc[0] for desc in mycursor.description]

# Create a pandas DataFrame
df = pd.DataFrame(results, columns=column_headers)

# Print the DataFrame
print(df)

# Close the cursor and connection
mycursor.close()
cnx.close()



      record_id audit_location  auditor_score  surveyor_score
0          5185      SoRu34980              1               1
1         59367      AkRu08112              3               3
2         37379      AkLu02044              0               0
3         51627      AkHa00421              3               3
4         28758      SoRu35221              0               0
...         ...            ...            ...             ...
1513      19733      SoRu36378              9               9
1514      42498      KiRu27180              6               6
1515       2716      HaRu16981              9               9
1516      41884      HaDe16326              1               1
1517      24662      HaRu20394              9               9

[1518 rows x 4 columns]


With the duplicates removed I now get 1518. What does this mean considering the auditor visited 1620 sites?
I think that is an excellent result. 1518/1620 = 94% of the records the auditor checked were correct

But that means that 102 records are incorrect. So let's look at those.

In [20]:
import mysql.connector
import pandas as pd

# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# Define the query to check mismatched scores, considering only the first visit
query_mismatch = """
    SELECT
        visits.record_id,
        auditor_report.location_id AS audit_location,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    WHERE
        auditor_report.true_water_source_score != water_quality.subjective_quality_score
        AND visits.visit_count = 1;
"""

# Execute the query for mismatches
mycursor.execute(query_mismatch)

# Fetch the results
results_mismatch = mycursor.fetchall()

# Get column names
column_names_mismatch = [desc[0] for desc in mycursor.description]

# Create a DataFrame
df_mismatch = pd.DataFrame(results_mismatch, columns=column_names_mismatch)

# Print the DataFrame
print("Records where surveyor's score does not match auditor's score (first visit only):")
print(df_mismatch)

# Check the count of mismatched records
print("\nNumber of mismatched records:", len(df_mismatch))

# Close the cursor and connection
mycursor.close()
cnx.close()


Records where surveyor's score does not match auditor's score (first visit only):
     record_id audit_location  auditor_score  surveyor_score
0        21160      AkRu05215              3              10
1         7938      KiRu29290              3              10
2        43140      KiHa22748              9              10
3        18495      SoRu37841              6              10
4        33931      KiRu27884              1              10
..         ...            ...            ...             ...
97       47831      AmPw12480              5              10
98       47055      AmRu14842              2              10
99       31888      AkRu03358              1              10
100      57735      AmRu13433              9              10
101      15929      AmAm09956              1              10

[102 rows x 4 columns]

Number of mismatched records: 102


Since we used some of this data in our previous analyses, we need to make sure those results are still valid, now we know some of them are
incorrect. We didn't use the scores that much, but we relied a lot on the type_of_water_source, so let's check if there are any errors there.

So, to do this, we need to grab the type_of_water_source column from the water_source table and call it survey_source, using the
source_id column to JOIN. Also select the type_of_water_source from the auditor_report table, and call it auditor_source.

In [35]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)
mycursor = cnx.cursor()

# Define the query to check discrepancies in type_of_water_source
query_type_discrepancies = """
    SELECT
    visits.record_id,
    auditor_report.location_id AS audit_location,
    auditor_report.type_of_water_source AS auditor_source,
    water_source.type_of_water_source AS survey_source,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_source ON visits.source_id = water_source.source_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
WHERE
     auditor_report.true_water_source_score != water_quality.subjective_quality_score AND
     visits.visit_count = 1;

"""

# Execute the query for type discrepancies
mycursor.execute(query_type_discrepancies)

# Fetch the results
results_type_discrepancies = mycursor.fetchall()

# Get column names
column_names_type_discrepancies = [desc[0] for desc in mycursor.description]

# Create a DataFrame
df_type_discrepancies = pd.DataFrame(results_type_discrepancies, columns=column_names_type_discrepancies)

# Print the DataFrame
print("Records where auditor's type of water source does not match survey's type of water source:")
print(df_type_discrepancies)

# Check the count of discrepancies
print("\nNumber of discrepancies in type of water source:", len(df_type_discrepancies))

# Close the cursor and connection
mycursor.close()
cnx.close()


Records where auditor's type of water source does not match survey's type of water source:
     record_id audit_location      auditor_source       survey_source  \
0        21160      AkRu05215                well                well   
1         7938      KiRu29290          shared_tap          shared_tap   
2        43140      KiHa22748  tap_in_home_broken  tap_in_home_broken   
3        18495      SoRu37841          shared_tap          shared_tap   
4        33931      KiRu27884                well                well   
..         ...            ...                 ...                 ...   
97       47831      AmPw12480          shared_tap          shared_tap   
98       47055      AmRu14842                well                well   
99       31888      AkRu03358                well                well   
100      57735      AmRu13433  tap_in_home_broken  tap_in_home_broken   
101      15929      AmAm09956                well                well   

     auditor_score  surveyor_sco

So what I can see is that the types of sources look the same! So even though the scores are wrong, the integrity of the type_of_water_source
data we analysed last time is not affected.

let's look at where these errors may have come from. At some of the locations, employees assigned scores incorrectly, and those records
ended up in this results set.

I think there are two reasons this can happen.
1. These workers are all humans and make mistakes so this is expected.
2. Unfortunately, the alternative is that someone assigned scores incorrectly on purpose!


In either case, the employees are the source of the errors, so let's JOIN the assigned_employee_id for all the people on our list from the visits
table to our query. Remember, our query shows the shows the 102 incorrect records, so when we join the employee data, we can see which
employees made these incorrect records.

In [36]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query
query = """
    SELECT
        visits.location_id,
        visits.record_id,
        visits.assigned_employee_id,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    JOIN
        employee ON visits.assigned_employee_id = employee.assigned_employee_id
    WHERE
        auditor_report.true_water_source_score != water_quality.subjective_quality_score
        AND visits.visit_count = 1;
"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


    location_id  record_id  assigned_employee_id  auditor_score  \
0     AkRu05215      21160                    34              3   
1     KiRu29290       7938                     1              3   
2     KiHa22748      43140                     1              9   
3     SoRu37841      18495                    34              6   
4     KiRu27884      33931                     1              1   
..          ...        ...                   ...            ...   
97    AmPw12480      47831                     5              5   
98    AmRu14842      47055                     1              2   
99    AkRu03358      31888                     3              1   
100   AmRu13433      57735                     3              9   
101   AmAm09956      15929                     3              1   

     surveyor_score  
0                10  
1                10  
2                10  
3                10  
4                10  
..              ...  
97               10  
98               10

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/997022841.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


So now we can link the incorrect records to the employees who recorded them. The ID's don't help us to identify them. We have employees' names
stored along with their IDs, so let's fetch their names from the employees table instead of the ID's.

In [39]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query
query = """
    SELECT
        visits.location_id,
        visits.record_id,
        employee.employee_name,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    JOIN
        employee ON visits.assigned_employee_id = employee.assigned_employee_id
    WHERE
        auditor_report.true_water_source_score != water_quality.subjective_quality_score
        AND visits.visit_count = 1;
"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


    location_id  record_id   employee_name  auditor_score  surveyor_score
0     AkRu05215      21160      Rudo Imani              3              10
1     KiRu29290       7938     Bello Azibo              3              10
2     KiHa22748      43140     Bello Azibo              9              10
3     SoRu37841      18495      Rudo Imani              6              10
4     KiRu27884      33931     Bello Azibo              1              10
..          ...        ...             ...            ...             ...
97    AmPw12480      47831  Zuriel Matembo              5              10
98    AmRu14842      47055     Bello Azibo              2              10
99    AkRu03358      31888  Malachi Mavuso              1              10
100   AmRu13433      57735  Malachi Mavuso              9              10
101   AmAm09956      15929  Malachi Mavuso              1              10

[102 rows x 5 columns]


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/603924149.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


Well this query is massive and complex, so maybe it is a good idea to save this as a CTE, so when we do more analysis, we can just call that CTE
like it was a table.

In [49]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query with CTE
query = """
    WITH Incorrect_records AS (
        SELECT
            visits.location_id,
            visits.record_id,
            employee.employee_name,
            auditor_report.true_water_source_score AS auditor_score,
            water_quality.subjective_quality_score AS surveyor_score
        FROM
            auditor_report
        JOIN
            visits ON auditor_report.location_id = visits.location_id
        JOIN
            water_quality ON visits.record_id = water_quality.record_id
        JOIN
            employee ON visits.assigned_employee_id = employee.assigned_employee_id
        WHERE
            auditor_report.true_water_source_score != water_quality.subjective_quality_score
            AND visits.visit_count = 1
    )
    select * from Incorrect_records;

"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


    location_id  record_id   employee_name  auditor_score  surveyor_score
0     AkRu05215      21160      Rudo Imani              3              10
1     KiRu29290       7938     Bello Azibo              3              10
2     KiHa22748      43140     Bello Azibo              9              10
3     SoRu37841      18495      Rudo Imani              6              10
4     KiRu27884      33931     Bello Azibo              1              10
..          ...        ...             ...            ...             ...
97    AmPw12480      47831  Zuriel Matembo              5              10
98    AmRu14842      47055     Bello Azibo              2              10
99    AkRu03358      31888  Malachi Mavuso              1              10
100   AmRu13433      57735  Malachi Mavuso              9              10
101   AmAm09956      15929  Malachi Mavuso              1              10

[102 rows x 5 columns]


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/2742686943.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


In [56]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
  user='root',
  password='Mayar123@',
  host='127.0.0.1',
  database='md_water_services',
  auth_plugin='mysql_native_password'
)

# Define the query to create the table 'incorrect' (modify schema as needed)
create_table_query = """
  CREATE TABLE IF NOT EXISTS incorrect_records(
    location_id VARCHAR(255),
    record_id INT,
    employee_name VARCHAR(255),
    auditor_score INT,
    surveyor_score INT
  );
"""

# Define the query to insert data
insert_query = """
  INSERT INTO incorrect_records (location_id, record_id, employee_name, auditor_score, surveyor_score)
  SELECT
    visits.location_id,
    visits.record_id,
    employee.employee_name,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score
  FROM
    auditor_report
  JOIN
    visits ON auditor_report.location_id = visits.location_id
  JOIN
    water_quality ON visits.record_id = water_quality.record_id
  JOIN
    employee ON visits.assigned_employee_id = employee.assigned_employee_id
  WHERE
    auditor_report.true_water_source_score != water_quality.subjective_quality_score
    AND visits.visit_count = 1;
"""

# Create a cursor object
cursor = cnx.cursor()

# Execute the CREATE TABLE query first
cursor.execute(create_table_query)
cnx.commit()

# Execute the INSERT query to populate the table
cursor.execute(insert_query)
cnx.commit()

# Print success message (optional)
print("Table 'incorrect' created and data inserted successfully!")

# Close the connection
cnx.close()


Table 'incorrect' created and data inserted successfully!


Now that we defined Incorrect_records, we can query it like any other table.

Let's first get a unique list of employees from this table.

In [57]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query with CTE
query = """
    
    select DISTINCT employee_name from Incorrect_records;

"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


     employee_name
0       Rudo Imani
1      Bello Azibo
2   Zuriel Matembo
3    Yewande Ebele
4    Jengo Tumaini
5        Farai Nia
6   Malachi Mavuso
7     Makena Thabo
8   Lalitha Kaburi
9      Gamba Shani
10     Enitan Zuri
11   Thandiwe Kito
12       Pili Zola
13       Usafi Ayo
14    Deka Osumare
15        Ona Sefu
16      Xola Uzuri


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/2390876984.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


I got 17 employees

let's calculate how many mistakes each employee made. So basically we want to count how many times their name is in
Incorrect_records list, and then group them by name

In [61]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query with CTE
query = """
    
    select employee_name,COUNT(*) AS number_of_mistakes from Incorrect_records 
    GROUP BY employee_name ORDER BY number_of_mistakes DESC;

"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


     employee_name  number_of_mistakes
0      Bello Azibo                  26
1   Malachi Mavuso                  21
2   Zuriel Matembo                  17
3   Lalitha Kaburi                   7
4       Rudo Imani                   5
5        Farai Nia                   4
6      Enitan Zuri                   4
7    Yewande Ebele                   3
8    Jengo Tumaini                   3
9     Makena Thabo                   3
10     Gamba Shani                   3
11   Thandiwe Kito                   1
12       Pili Zola                   1
13       Usafi Ayo                   1
14    Deka Osumare                   1
15        Ona Sefu                   1
16      Xola Uzuri                   1


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/1696243433.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


It looks like some of our surveyors are making a lot of "mistakes" while many of the other surveyors are only making a few.

Ok, so thinking about this a bit. How would we go about finding out if any of our employees are corrupt?

Let's say all employees make mistakes, if someone is corrupt, they will be making a lot of "mistakes", more than average, for example. But someone
could just be clumsy, so we should try to get more evidence...

Our auditor did say some of the things he heard on the streets were quite shady, and he recorded this in the statements column. Considering
both of these sources should give us a pretty reliable answer.

# Gathering Evidence

So let's try to find all of the employees who have an above-average number of mistakes.

In [64]:


# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query to create the Incorrect_records CTE and get the data
query = """
WITH incorrect_records AS (
    SELECT
        visits.location_id,
        visits.record_id,
        employee.employee_name,
        auditor_report.true_water_source_score AS auditor_score,
        water_quality.subjective_quality_score AS surveyor_score
    FROM
        auditor_report
    JOIN
        visits ON auditor_report.location_id = visits.location_id
    JOIN
        water_quality ON visits.record_id = water_quality.record_id
    JOIN
        employee ON visits.assigned_employee_id = employee.assigned_employee_id
    WHERE
        auditor_report.true_water_source_score != water_quality.subjective_quality_score
        AND visits.visit_count = 1
)
SELECT
    employee_name,
    COUNT(*) AS number_of_mistakes
FROM
    incorrect_records
GROUP BY
    employee_name;
"""

# Execute the query and load the data into a Pandas DataFrame
error_count_df = pd.read_sql(query, con=cnx)

# Calculate the average number of mistakes
avg_error_count = error_count_df['number_of_mistakes'].mean()

# Get the suspect list
suspect_list = error_count_df[error_count_df['number_of_mistakes'] > avg_error_count]

# Print the suspect list
print("Employees with above-average mistakes:")
print(suspect_list)

# Close the connection
cnx.close()


Employees with above-average mistakes:
    employee_name  number_of_mistakes
1     Bello Azibo                  26
2  Zuriel Matembo                  17
6  Malachi Mavuso                  21
8  Lalitha Kaburi                   7


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/2732542279.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  error_count_df = pd.read_sql(query, con=cnx)


Let's start by cleaning up our code a bit. First, Incorrect_records is a result we'll be using for the rest of the analysis, but it makes the
query a bit less readable. So, let's convert it to a VIEW. We can then use it as if it was a table. It will make our code much simpler to read, but, it
comes at a cost. We can add comments to CTEs in our code, so if we return to that query a year later, we can read those comments and quickly
understand what Incorrect_records represents. If we save it as a VIEW, it is not as obvious. So we should add comments in places where we
use Incorrect_records.

So, replace WITH with CREATE VIEW like this, and I added the statements column

In [69]:

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Create a cursor object
cursor = cnx.cursor()

# Drop the view if it exists
drop_view_query = "DROP VIEW IF EXISTS Incorrect_records;"
cursor.execute(drop_view_query)

# Define the query to create the Incorrect_records view
create_view_query = """
CREATE VIEW Incorrect_records AS
SELECT
    auditor_report.location_id,
    visits.record_id,
    employee.employee_name,
    auditor_report.true_water_source_score AS auditor_score,
    wq.subjective_quality_score AS surveyor_score,
    auditor_report.statements AS statements
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality AS wq ON visits.record_id = wq.record_id
JOIN
    employee ON employee.assigned_employee_id = visits.assigned_employee_id
WHERE
    visits.visit_count = 1
    AND auditor_report.true_water_source_score != wq.subjective_quality_score;
"""
cursor.execute(create_view_query)

# Define the query to get the data from the Incorrect_records view
query = """
SELECT
    employee_name,
    COUNT(*) AS number_of_mistakes
FROM
    Incorrect_records
GROUP BY
    employee_name;
"""

# Execute the query and load the data into a Pandas DataFrame
error_count_df = pd.read_sql(query, con=cnx)

# Calculate the average number of mistakes
avg_error_count = error_count_df['number_of_mistakes'].mean()

# Get the suspect list
suspect_list = error_count_df[error_count_df['number_of_mistakes'] > avg_error_count]

# Print the suspect list
print("Employees with above-average mistakes:")
print(suspect_list)

# Close the cursor and connection
cursor.close()
cnx.close()


Employees with above-average mistakes:
    employee_name  number_of_mistakes
1     Bello Azibo                  26
2  Zuriel Matembo                  17
6  Malachi Mavuso                  21
8  Lalitha Kaburi                   7


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/1727823294.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  error_count_df = pd.read_sql(query, con=cnx)


In [70]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query with CTE
query = """
    SELECT * FROM Incorrect_records;
"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


    location_id  record_id   employee_name  auditor_score  surveyor_score  \
0     AkRu05215      21160      Rudo Imani              3              10   
1     KiRu29290       7938     Bello Azibo              3              10   
2     KiHa22748      43140     Bello Azibo              9              10   
3     SoRu37841      18495      Rudo Imani              6              10   
4     KiRu27884      33931     Bello Azibo              1              10   
..          ...        ...             ...            ...             ...   
97    AmPw12480      47831  Zuriel Matembo              5              10   
98    AmRu14842      47055     Bello Azibo              2              10   
99    AkRu03358      31888  Malachi Mavuso              1              10   
100   AmRu13433      57735  Malachi Mavuso              9              10   
101   AmAm09956      15929  Malachi Mavuso              1              10   

                                            statements  
0    Villagers adm

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/2537148912.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


Next, we convert the query error_count, we made earlier, into a CTE. Test it to make sure it gives the same result again, using SELECT * FROM
Incorrect_records. On large queries like this, it is better to build the query, and test each step, because fixing errors becomes harder as the
query grows.

In [72]:


# Establish a connection with the specified auth_plugin
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query with CTE
query = """
    WITH error_count AS ( -- This CTE calculates the number of mistakes each employee made
SELECT
employee_name,
COUNT(employee_name) AS number_of_mistakes
FROM
Incorrect_records

GROUP BY
employee_name)
SELECT * FROM error_count;
"""

# Execute the query and load into a pandas DataFrame
df = pd.read_sql(query, con=cnx)

# Print the DataFrame
print(df)

# Close the connection
cnx.close()


     employee_name  number_of_mistakes
0       Rudo Imani                   5
1      Bello Azibo                  26
2   Zuriel Matembo                  17
3    Yewande Ebele                   3
4    Jengo Tumaini                   3
5        Farai Nia                   4
6   Malachi Mavuso                  21
7     Makena Thabo                   3
8   Lalitha Kaburi                   7
9      Gamba Shani                   3
10     Enitan Zuri                   4
11   Thandiwe Kito                   1
12       Pili Zola                   1
13       Usafi Ayo                   1
14    Deka Osumare                   1
15        Ona Sefu                   1
16      Xola Uzuri                   1


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/3980056284.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=cnx)


calculating the average of the number_of_mistakes in error_count.

In [73]:


# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Create a cursor object
cursor = cnx.cursor()

# Drop the view if it exists


# Define the query to calculate the average number of mistakes
query = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
)
SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count;
"""

# Execute the query and fetch the result
cursor.execute(query)
avg_error_count = cursor.fetchone()[0]

# Print the average error count
print(f"\nAverage Number of Mistakes: {avg_error_count}")

# Close the cursor and connection
cursor.close()
cnx.close()



Average Number of Mistakes: 6.0000


To find the employees who made more mistakes than the average person, we need the employee's names, the number of mistakes each one
made, and filter the employees with an above-average number of mistakes.

In [74]:

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Create a cursor object
cursor = cnx.cursor()


# Define the query to find employees with above-average mistakes
query = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
),
avg_error AS (
    SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count
)
SELECT
    ec.employee_name,
    ec.number_of_mistakes
FROM
    error_count ec,
    avg_error ae
WHERE
    ec.number_of_mistakes > ae.avg_error_count_per_empl;
"""

# Execute the query and load the results into a pandas DataFrame
suspect_list_df = pd.read_sql(query, con=cnx)

# Print the suspect list
print("Employees with above-average mistakes:")
print(suspect_list_df)

# Close the cursor and connection
cursor.close()
cnx.close()


Employees with above-average mistakes:
    employee_name  number_of_mistakes
0     Bello Azibo                  26
1  Zuriel Matembo                  17
2  Malachi Mavuso                  21
3  Lalitha Kaburi                   7


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/645360838.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  suspect_list_df = pd.read_sql(query, con=cnx)


These are the employees who made more mistakes, on average, than their peers.

We should look at the Incorrect_records table again, and isolate all of the records these four employees gathered. We should also look at the
statements for these records to look for patterns.

In [75]:


# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)




# Define the query to get the records of the suspect employees
query = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
),
avg_error AS (
    SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count
),
suspect_list AS (
    SELECT
        ec.employee_name,
        ec.number_of_mistakes
    FROM
        error_count ec,
        avg_error ae
    WHERE
        ec.number_of_mistakes > ae.avg_error_count_per_empl
)
SELECT
    ir.location_id,
    ir.record_id,
    ir.employee_name,
    ir.auditor_score,
    ir.surveyor_score,
    ir.statements
FROM
    Incorrect_records ir
JOIN
    suspect_list sl ON ir.employee_name = sl.employee_name;
"""

# Execute the query and load the results into a pandas DataFrame
suspect_records_df = pd.read_sql(query, con=cnx)

# Print the suspect records
print("Records of employees with above-average mistakes:")
print(suspect_records_df)

# Close the cursor and connection
cursor.close()
cnx.close()


Records of employees with above-average mistakes:
   location_id  record_id   employee_name  auditor_score  surveyor_score  \
0    AmRu14842      47055     Bello Azibo              2              10   
1    HaYa21395      28729     Bello Azibo              2              10   
2    AkRu05741      19961     Bello Azibo              3              10   
3    AmRu14887      31028     Bello Azibo              9              10   
4    KiMr24919      23328     Bello Azibo              1              10   
..         ...        ...             ...            ...             ...   
66   SoIl32770       7548  Lalitha Kaburi              0              10   
67   AkRu07310      37457  Lalitha Kaburi              9              10   
68   AkRu04935      16410  Lalitha Kaburi              3              10   
69   SoKo33094      16159  Lalitha Kaburi              0              10   
70   KiIs24083      28404  Lalitha Kaburi              2              10   

                                     

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/167050977.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  suspect_records_df = pd.read_sql(query, con=cnx)


let's add the statements column to the Incorrect_records view. Then pull up all of the records where the employee_name is in the
suspect list.

In [78]:


# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)



# Define the query to get the suspect records
query = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
),
avg_error AS (
    SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count
),
suspect_list AS (
    SELECT
        ec.employee_name,
        ec.number_of_mistakes
    FROM
        error_count ec,
        avg_error ae
    WHERE
        ec.number_of_mistakes > ae.avg_error_count_per_empl
)
SELECT
    ir.employee_name,
    ir.location_id,
    ir.statements
FROM
    Incorrect_records ir
WHERE
    ir.employee_name IN (SELECT employee_name FROM suspect_list);
"""

# Execute the query and load the results into a pandas DataFrame
suspect_records_df = pd.read_sql(query, con=cnx)

# Print the suspect records
print("Records of employees with above-average mistakes:")
print(suspect_records_df)

# Close the cursor and connection
cursor.close()
cnx.close()


Records of employees with above-average mistakes:
     employee_name location_id  \
0      Bello Azibo   KiRu29290   
1      Bello Azibo   KiHa22748   
2      Bello Azibo   KiRu27884   
3   Zuriel Matembo   KiZu31170   
4      Bello Azibo   AkRu06495   
..             ...         ...   
66  Zuriel Matembo   AmPw12480   
67     Bello Azibo   AmRu14842   
68  Malachi Mavuso   AkRu03358   
69  Malachi Mavuso   AmRu13433   
70  Malachi Mavuso   AmAm09956   

                                           statements  
0   A young artist sketches the faces in the queue...  
1   A young girl's hopeful eyes are clouded by mis...  
2   A traditional healer's empathy turns to bitter...  
3   A community leader stood with his people, expr...  
4   A healthcare worker in the queue expressed fea...  
..                                                ...  
66  A village chief's dignity is marred by frustra...  
67  An elderly woman's weary eyes reflect the toll...  
68  A traveling trader's pause in the

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/820962457.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  suspect_records_df = pd.read_sql(query, con=cnx)


If you have a look, you will notice some alarming statements about these four officials (look at these records: AkRu04508, AkRu07310,
KiRu29639, AmAm09607, for example. See how the word "cash" is used a lot in these statements.

we will filter the records tha have 'cash'

In [80]:
import pandas as pd
import mysql.connector

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define the query to get records mentioning "cash"
query = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
),
avg_error AS (
    SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count
),
suspect_list AS (
    SELECT
        ec.employee_name,
        ec.number_of_mistakes
    FROM
        error_count ec,
        avg_error ae
    WHERE
        ec.number_of_mistakes > ae.avg_error_count_per_empl
)
SELECT
    ir.employee_name,
    ir.location_id,
    ir.statements
FROM
    Incorrect_records ir
WHERE
    ir.employee_name IN (SELECT employee_name FROM suspect_list)
    AND ir.statements LIKE '%cash%';
"""

# Execute the query and load the results into a pandas DataFrame
cash_related_records_df = pd.read_sql(query, con=cnx)

# Print the filtered records
print("Records of employees with above-average mistakes that mention 'cash':")
print(cash_related_records_df)

# Close the connection
cnx.close()


Records of employees with above-average mistakes that mention 'cash':
     employee_name location_id  \
0      Bello Azibo   AkRu05741   
1      Bello Azibo   AmRu14887   
2      Bello Azibo   KiMr24919   
3      Bello Azibo   KiRu29639   
4      Bello Azibo   AkRu04508   
5      Bello Azibo   KiRu27065   
6      Bello Azibo   HaSe21323   
7      Bello Azibo   KiIs23853   
8   Zuriel Matembo   HaRu20146   
9   Zuriel Matembo   HaSe20888   
10  Zuriel Matembo   SoIl32575   
11  Zuriel Matembo   AkRu05880   
12  Zuriel Matembo   SoRu38331   
13  Malachi Mavuso   AmAm09956   
14  Malachi Mavuso   AmRu15719   
15  Malachi Mavuso   KiRu25347   
16  Malachi Mavuso   AmAm09607   
17  Lalitha Kaburi   KiRu29329   
18  Lalitha Kaburi   AkRu07310   

                                           statements  
0   An air of mistrust surrounded the official, as...  
1   Villagers expressed their discomfort with an o...  
2   Suspicion and unease colored the villagers' ac...  
3   An unsettling atmosph

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/3984398263.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cash_related_records_df = pd.read_sql(query, con=cnx)


Check if there are any employees in the Incorrect_records table with statements mentioning "cash" that are not in our suspect list.

In [82]:

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(
    user='root', 
    password='Mayar123@', 
    host='127.0.0.1', 
    database='md_water_services',
    auth_plugin='mysql_native_password'
)

# Define queries
query_incorrect_records = """
SELECT *
FROM Incorrect_records;
"""

query_suspect_list = """
WITH error_count AS (
    SELECT
        employee_name,
        COUNT(employee_name) AS number_of_mistakes
    FROM
        Incorrect_records
    GROUP BY
        employee_name
),
avg_error AS (
    SELECT AVG(number_of_mistakes) AS avg_error_count_per_empl FROM error_count
),
suspect_list AS (
    SELECT
        ec.employee_name,
        ec.number_of_mistakes
    FROM
        error_count ec,
        avg_error ae
    WHERE
        ec.number_of_mistakes > ae.avg_error_count_per_empl
)
SELECT employee_name
FROM suspect_list;
"""

# Execute the queries and load into Pandas DataFrames
incorrect_records_df = pd.read_sql(query_incorrect_records, con=cnx)
suspect_list_df = pd.read_sql(query_suspect_list, con=cnx)

# Get the list of suspect employees
suspect_list = suspect_list_df['employee_name'].tolist()

# Filter records mentioning 'cash'
cash_records_df = incorrect_records_df[incorrect_records_df['statements'].str.contains('cash', case=False, na=False)]

# Exclude records from suspect employees
non_suspect_cash_records_df = cash_records_df[~cash_records_df['employee_name'].isin(suspect_list)]

# Print the results
print("Records mentioning 'cash' not from suspect employees:")
print(non_suspect_cash_records_df)

# Close the connection
cnx.close()


Records mentioning 'cash' not from suspect employees:
Empty DataFrame
Columns: [location_id, record_id, employee_name, auditor_score, surveyor_score, statements]
Index: []


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/4003339870.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  incorrect_records_df = pd.read_sql(query_incorrect_records, con=cnx)
/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_8068/4003339870.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  suspect_list_df = pd.read_sql(query_suspect_list, con=cnx)


I get an empty result, so no one, except the four suspects, has these allegations of bribery.

So we can sum up the evidence we have for Zuriel Matembo, Malachi Mavuso, Bello Azibo and Lalitha Kaburi:
1. They all made more mistakes than their peers on average.
2. They all have incriminating statements made against them, and only them.
Keep in mind, that this is not decisive proof, but it is concerning enough that we should flag it. Pres. Naledi has worked hard to stamp out
corruption, so she would urge us to report this.

# Starting the final journey

Let's summarise the data we need, and where to find it:
• All of the information about the location of a water source is in the location table, specifically the town and province of that water source.
• water_source has the type of source and the number of people served by each source.
• visits has queue information, and connects source_id to location_id. There were multiple visits to sites, so we need to be careful to
include duplicate data (visit_count > 1 ).
• well_pollution has information about the quality of water from only wells, so we need to keep that in mind when we join this table.


Previously, we couldn't link provinces and towns to the type of water sources, the number of people served by those sources, queue times, or pol-
lution data, but we can now.

Things that spring to mind for me:
1. Are there any specific provinces, or towns where some sources are more abundant?
2. We identified that tap_in_home_broken taps are easy wins. Are there any towns where this is a particular problem?

To answer question 1, we will need province_name and town_name from the location table. We also need to know type_of_water_source and
number_of_people_served from the water_source table.


The problem is that the location table uses location_id while water_source only has source_id. So we won't be able to join these tables di-
rectly. But the visits table maps location_id and source_id. So if we use visits as the table we query from, we can join location where

the location_id matches, and water_source where the source_id matches.


Before we can analyse, we need to assemble data into a table first. It is quite complex, but once we're done, the analysis is much simpler!

Start by joining location to visits.

then, we can join the water_source table on the key shared between water_source and visits.

In [9]:

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' 
)

# SQL query to join the tables and get the required data
query = '''
WITH location_visits AS (
    SELECT 
        loc.province_name,
        loc.town_name,
        COUNT(v.record_id) AS visit_count,
        loc.location_id
    FROM 
        location loc
    JOIN 
        visits v ON loc.location_id = v.location_id
    GROUP BY 
        loc.province_name, loc.town_name, loc.location_id
)
SELECT 
    lv.province_name,
    lv.town_name,
    lv.visit_count,
    lv.location_id,
    ws.type_of_water_source,
    ws.number_of_people_served
FROM 
    location_visits lv
JOIN 
    visits v ON lv.location_id = v.location_id
JOIN 
    water_source ws ON v.source_id = ws.source_id

ORDER BY 
    lv.province_name, lv.town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/2629917947.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


      province_name town_name  visit_count location_id type_of_water_source  \
0            Akatsi    Harare            1   AkHa00000          tap_in_home   
1            Akatsi    Harare            1   AkHa00001   tap_in_home_broken   
2            Akatsi    Harare            1   AkHa00002   tap_in_home_broken   
3            Akatsi    Harare            1   AkHa00003                 well   
4            Akatsi    Harare            1   AkHa00004   tap_in_home_broken   
...             ...       ...          ...         ...                  ...   
60141        Sokoto     Rural            1   SoRu39645          tap_in_home   
60142        Sokoto     Rural            1   SoRu39646          tap_in_home   
60143        Sokoto     Rural            1   SoRu39647                 well   
60144        Sokoto     Rural            1   SoRu39648   tap_in_home_broken   
60145        Sokoto     Rural            1   SoRu39649                 well   

       number_of_people_served  
0                 

Note that there are rows where visit_count > 1. These were the sites our surveyors collected additional information for, but they happened at the
same source/location. For example, add this to your query: WHERE visits.location_id = 'AkHa00103'

In [10]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to join the tables and get the required data
query = '''
WITH location_visits AS (
    SELECT 
        loc.province_name,
        loc.town_name,
        COUNT(v.record_id) AS visit_count,
        loc.location_id
    FROM 
        location loc
    JOIN 
        visits v ON loc.location_id = v.location_id
    WHERE
        v.location_id = 'AkHa00103'
    GROUP BY 
        loc.province_name, loc.town_name, loc.location_id
)
SELECT 
    lv.province_name,
    lv.town_name,
    lv.visit_count,
    ws.type_of_water_source,
    ws.number_of_people_served
FROM 
    location_visits lv
JOIN 
    visits v ON lv.location_id = v.location_id
JOIN 
    water_source ws ON v.source_id = ws.source_id
ORDER BY 
    lv.province_name, lv.town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


  province_name town_name  visit_count type_of_water_source  \
0        Akatsi    Harare            8           shared_tap   
1        Akatsi    Harare            8           shared_tap   
2        Akatsi    Harare            8           shared_tap   
3        Akatsi    Harare            8           shared_tap   
4        Akatsi    Harare            8           shared_tap   
5        Akatsi    Harare            8           shared_tap   
6        Akatsi    Harare            8           shared_tap   
7        Akatsi    Harare            8           shared_tap   

   number_of_people_served  
0                     3340  
1                     3340  
2                     3340  
3                     3340  
4                     3340  
5                     3340  
6                     3340  
7                     3340  


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/2660195176.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


There you can see what I mean. For one location, there are multiple AkHa00103 records for the same location. If we aggregate, we will include
these rows, so our results will be incorrect. To fix this, we can just select rows where visits.visit_count = 1.

Remove WHERE visits.location_id = 'AkHa00103' and add the visits.visit_count = 1 as a filter.

In [11]:

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to join the tables and get the required data, filtering by visit_count = 1
query = '''
WITH location_visits AS (
    SELECT 
        loc.province_name,
        loc.town_name,
        COUNT(v.record_id) AS visit_count,
        loc.location_id
    FROM 
        location loc
    JOIN 
        visits v ON loc.location_id = v.location_id
    WHERE
        v.visit_count = 1
    GROUP BY 
        loc.province_name, loc.town_name, loc.location_id
)
SELECT 
    lv.province_name,
    lv.town_name,
    lv.visit_count,
    ws.type_of_water_source,
    ws.number_of_people_served
FROM 
    location_visits lv
JOIN 
    visits v ON lv.location_id = v.location_id
JOIN 
    water_source ws ON v.source_id = ws.source_id
ORDER BY 
    lv.province_name, lv.town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3649098336.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


      province_name town_name  visit_count type_of_water_source  \
0            Akatsi    Harare            1          tap_in_home   
1            Akatsi    Harare            1                 well   
2            Akatsi    Harare            1           shared_tap   
3            Akatsi    Harare            1          tap_in_home   
4            Akatsi    Harare            1                 well   
...             ...       ...          ...                  ...   
60141        Sokoto     Rural            1                 well   
60142        Sokoto     Rural            1                river   
60143        Sokoto     Rural            1                river   
60144        Sokoto     Rural            1                river   
60145        Sokoto     Rural            1          tap_in_home   

       number_of_people_served  
0                          568  
1                          386  
2                          854  
3                          838  
4                          258

Ok, now that we verified that the table is joined correctly, we can remove the location_id and visit_count columns.

In [13]:
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)
# SQL query to join the tables and get the required data, filtering by visit_count = 1
query = '''
WITH location_visits AS (
    SELECT 
        loc.province_name,
        loc.town_name,
        loc.location_id
    FROM 
        location loc
    JOIN 
        visits v ON loc.location_id = v.location_id
    WHERE
        v.visit_count = 1
    GROUP BY 
        loc.province_name, loc.town_name, loc.location_id
)
SELECT 
    lv.province_name,
    lv.town_name,
    ws.type_of_water_source,
    ws.number_of_people_served
FROM 
    location_visits lv
JOIN 
    visits v ON lv.location_id = v.location_id
JOIN 
    water_source ws ON v.source_id = ws.source_id
ORDER BY 
    lv.province_name, lv.town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/892407237.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


      province_name town_name type_of_water_source  number_of_people_served
0            Akatsi    Harare   tap_in_home_broken                      578
1            Akatsi    Harare   tap_in_home_broken                      556
2            Akatsi    Harare          tap_in_home                      418
3            Akatsi    Harare                 well                      224
4            Akatsi    Harare                 well                      188
...             ...       ...                  ...                      ...
60141        Sokoto     Rural   tap_in_home_broken                      522
60142        Sokoto     Rural          tap_in_home                      870
60143        Sokoto     Rural                 well                      334
60144        Sokoto     Rural                river                      930
60145        Sokoto     Rural                 well                      242

[60146 rows x 4 columns]


Add the location_type column from location and time_in_queue from visits to our results set.

In [26]:

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to join the tables and get the required data, filtering by visit_count = 1
query = '''
SELECT
water_source.type_of_water_source,
location.town_name,
location.province_name,
location.location_type,
water_source.number_of_people_served,
visits.time_in_queue
FROM
visits
INNER JOIN
location
ON location.location_id = visits.location_id
INNER JOIN
water_source
ON water_source.source_id = visits.source_id
WHERE
visits.visit_count = 1;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3606101376.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


      type_of_water_source town_name province_name location_type  \
0                    river    Ilanga        Sokoto         Urban   
1                     well     Rural      Kilimani         Rural   
2               shared_tap     Rural       Hawassa         Rural   
3                     well    Lusaka        Akatsi         Urban   
4               shared_tap     Rural        Akatsi         Rural   
...                    ...       ...           ...           ...   
39645          tap_in_home    Dahabu        Amanzi         Urban   
39646           shared_tap     Rural      Kilimani         Rural   
39647   tap_in_home_broken     Rural        Amanzi         Rural   
39648          tap_in_home    Mrembo      Kilimani         Urban   
39649                 well     Rural        Akatsi         Rural   

       number_of_people_served  time_in_queue  
0                          402             15  
1                          252              0  
2                          542         

Last one! Now we need to grab the results from the well_pollution table.
This one is a bit trickier. The well_pollution table contained only data for well. If we just use JOIN, we will do an inner join, so that only records
that are in well_pollution AND visits will be joined. We have to use a LEFT JOIN to join theresults from the well_pollution table for well
sources, and will be NULL for all of the rest. 

In [22]:

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',    # e.g., 'root'
    password='Mayar123@',# e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to join the tables and get the required data, filtering by visit_count = 1
query = '''
SELECT
water_source.type_of_water_source,
location.town_name,
location.province_name,
location.location_type,
water_source.number_of_people_served,
visits.time_in_queue,
well_pollution.results
FROM
visits
LEFT JOIN
well_pollution
ON well_pollution.source_id = visits.source_id
INNER JOIN
location
ON location.location_id = visits.location_id
INNER JOIN
water_source
ON water_source.source_id = visits.source_id
WHERE
visits.visit_count = 1;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3244186490.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


      type_of_water_source town_name province_name location_type  \
0                    river    Ilanga        Sokoto         Urban   
1                     well     Rural      Kilimani         Rural   
2               shared_tap     Rural       Hawassa         Rural   
3                     well    Lusaka        Akatsi         Urban   
4               shared_tap     Rural        Akatsi         Rural   
...                    ...       ...           ...           ...   
39645          tap_in_home    Dahabu        Amanzi         Urban   
39646           shared_tap     Rural      Kilimani         Rural   
39647   tap_in_home_broken     Rural        Amanzi         Rural   
39648          tap_in_home    Mrembo      Kilimani         Urban   
39649                 well     Rural        Akatsi         Rural   

       number_of_people_served  time_in_queue                   results  
0                          402             15                      None  
1                          252     

So this table contains the data we need for this analysis. Now we want to analyse the data in the results set. We can either create a CTE, and then
query it, or in my case, I'll make it a VIEW so it is easier to share with you. I'll call it the combined_analysis_table.

In [28]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to create the view
create_view_query = '''
CREATE OR REPLACE VIEW combined_analysis_table AS
SELECT
    water_source.type_of_water_source AS source_type,
    location.town_name,
    location.province_name,
    location.location_type,
    water_source.number_of_people_served AS people_served,
    visits.time_in_queue,
    well_pollution.results
FROM
    visits
LEFT JOIN
    well_pollution ON well_pollution.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
INNER JOIN
    water_source ON water_source.source_id = visits.source_id
WHERE
    visits.visit_count = 1;
'''

# Execute the query to create the view
cursor.execute(create_view_query)

# Commit the transaction
conn.commit()

# SQL query to get the data from the view
query = '''
SELECT
    source_type,
    town_name,
    province_name,
    location_type,
    people_served,
    time_in_queue,
    results
FROM
    combined_analysis_table
ORDER BY
    province_name, town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the cursor and the database connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/1037038600.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


              source_type town_name province_name location_type  \
0                    well    Harare        Akatsi         Urban   
1             tap_in_home    Harare        Akatsi         Urban   
2              shared_tap    Harare        Akatsi         Urban   
3                    well    Harare        Akatsi         Urban   
4             tap_in_home    Harare        Akatsi         Urban   
...                   ...       ...           ...           ...   
39645  tap_in_home_broken     Rural        Sokoto         Rural   
39646         tap_in_home     Rural        Sokoto         Rural   
39647          shared_tap     Rural        Sokoto         Rural   
39648         tap_in_home     Rural        Sokoto         Rural   
39649                well     Rural        Sokoto         Rural   

       people_served  time_in_queue                   results  
0                386              0                     Clean  
1                838              0                      None  
2  

This view creates a "table" that pulls all of the important information from different tables into one.  our query is starting to slow
down because it involves a lot of steps, and runs on 60000 rows of data.

# Finding the final insights from our data

We're building another pivot table! This time, we want to break down our data into provinces or towns and source types. If we understand where
the problems are, and what we need to improve at those locations, we can make an informed decision on where to send our repair teams.

In [29]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to create the view
create_view_query = '''
CREATE OR REPLACE VIEW combined_analysis_table AS
SELECT
    water_source.type_of_water_source AS source_type,
    location.town_name,
    location.province_name,
    location.location_type,
    water_source.number_of_people_served AS people_served,
    visits.time_in_queue,
    well_pollution.results
FROM
    visits
LEFT JOIN
    well_pollution ON well_pollution.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
INNER JOIN
    water_source ON water_source.source_id = visits.source_id
WHERE
    visits.visit_count = 1;
'''

# Execute the query to create the view
cursor.execute(create_view_query)

# Commit the transaction
conn.commit()

# SQL query to get the data from the view
query = '''
-- This CTE calculates the population of each province
WITH province_totals AS (
    SELECT
        province_name,                       # Select province name
        SUM(people_served) AS total_ppl_serv # Calculate the total number of people served in each province
    FROM
        combined_analysis_table              # From the combined_analysis_table
    GROUP BY
        province_name                        # Group by province name
)
SELECT
    ct.province_name,                        # Select province name from combined_analysis_table
    -- The following case statements create columns for each type of water source.
    -- The results are aggregated and percentages are calculated.
    ROUND(
        (SUM(CASE WHEN source_type = 'river' THEN people_served ELSE 0 END) * 100.0 / pt.total_ppl_serv), 
        0
    ) AS river,                              # Calculate the percentage of people served by river
    ROUND(
        (SUM(CASE WHEN source_type = 'shared_tap' THEN people_served ELSE 0 END) * 100.0 / pt.total_ppl_serv), 
        0
    ) AS shared_tap,                         # Calculate the percentage of people served by shared tap
    ROUND(
        (SUM(CASE WHEN source_type = 'tap_in_home' THEN people_served ELSE 0 END) * 100.0 / pt.total_ppl_serv), 
        0
    ) AS tap_in_home,                        # Calculate the percentage of people served by tap in home
    ROUND(
        (SUM(CASE WHEN source_type = 'tap_in_home_broken' THEN people_served ELSE 0 END) * 100.0 / pt.total_ppl_serv), 
        0
    ) AS tap_in_home_broken,                 # Calculate the percentage of people served by broken tap in home
    ROUND(
        (SUM(CASE WHEN source_type = 'well' THEN people_served ELSE 0 END) * 100.0 / pt.total_ppl_serv), 
        0
    ) AS well                                # Calculate the percentage of people served by well
FROM
    combined_analysis_table ct               # From the combined_analysis_table aliased as ct
JOIN
    province_totals pt                       # Join with province_totals aliased as pt
    ON ct.province_name = pt.province_name   # On matching province names
GROUP BY
    ct.province_name                         # Group by province name
ORDER BY
    ct.province_name;                        # Order results by province name

'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the cursor and the database connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3954310581.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


  province_name  river  shared_tap  tap_in_home  tap_in_home_broken  well
0        Akatsi    5.0        49.0         14.0                10.0  23.0
1        Amanzi    3.0        38.0         28.0                24.0   7.0
2       Hawassa    4.0        43.0         15.0                15.0  24.0
3      Kilimani    8.0        47.0         13.0                12.0  20.0
4        Sokoto   21.0        38.0         16.0                10.0  15.0


Look at the river column, Sokoto has the largest population of people drinking river water. We should send our drilling equipment to Sokoto
first, so people can drink safe filtered water from a well.

The majority of water from Amanzi comes from taps, but half of these home taps don't work because the infrastructure is broken. We need to
send out engineering teams to look at the infrastructure in Amanzi first. Fixing a large pump, treatment plant or reservoir means that
thousands of people will have running water. This means they will also not have to queue for water, so we improve two things at once.

In [32]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to create the province_totals CTE and select from it
query = '''
-- This CTE calculates the population of each province
WITH province_totals AS (
    SELECT
        province_name,                       -- Select province name
        SUM(people_served) AS total_ppl_serv -- Calculate the total number of people served in each province
    FROM
        combined_analysis_table              -- From the combined_analysis_table
    GROUP BY
        province_name                        -- Group by province name
)
SELECT
    *                                       -- Select all columns from province_totals
FROM
    province_totals;                        -- From the CTE province_totals
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/309895605.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


  province_name  total_ppl_serv
0        Sokoto       5774434.0
1      Kilimani       6584764.0
2       Hawassa       3843810.0
3        Akatsi       5993306.0
4        Amanzi       5431826.0


Let's aggregate the data per town now. You might think this is simple, but one little town makes this hard. Recall that there are two towns in Maji

Ndogo called Harare. One is in Akatsi, and one is in Kilimani. Amina is another example. So when we just aggregate by town, SQL doesn't distin-
guish between the different Harare's, so it combines their results.

To get around that, we have to group by province first, then by town, so that the duplicate towns are distinct because they are in different towns.

In [36]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to create the province_totals CTE and select from it
query = '''
WITH town_totals AS  (
    SELECT province_name, town_name, SUM(people_served) AS total_ppl_serv
    FROM combined_analysis_table
    GROUP BY province_name, town_name
)
SELECT
    ct.province_name,
    ct.town_name,
    ROUND((SUM(CASE WHEN source_type = 'river'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS river,
    ROUND((SUM(CASE WHEN source_type = 'shared_tap'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS shared_tap,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home_broken'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home_broken,
    ROUND((SUM(CASE WHEN source_type = 'well'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS well
FROM
    combined_analysis_table ct
JOIN
    town_totals tt ON ct.province_name = tt.province_name AND ct.town_name = tt.town_name
GROUP BY
    ct.province_name,
    ct.town_name
ORDER BY
    ct.town_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/1981473863.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


   province_name town_name  river  shared_tap  tap_in_home  \
0         Amanzi   Abidjan    2.0        53.0         22.0   
1       Kilimani     Amara    8.0        22.0         25.0   
2        Hawassa     Amina    2.0        14.0         19.0   
3         Amanzi     Amina    8.0        24.0          3.0   
4         Amanzi    Asmara    3.0        49.0         24.0   
5         Sokoto    Bahari   21.0        11.0         36.0   
6         Amanzi     Bello    3.0        53.0         20.0   
7         Sokoto    Cheche   19.0        16.0         35.0   
8         Amanzi    Dahabu    3.0        37.0         55.0   
9        Hawassa      Deka    3.0        16.0         23.0   
10       Hawassa    Djenne    3.0        18.0         19.0   
11        Akatsi    Harare    2.0        17.0         28.0   
12      Kilimani    Harare    7.0        11.0         30.0   
13        Sokoto    Ilanga   16.0        12.0         36.0   
14      Kilimani   Isiqalo    7.0        19.0         25.0   
15      

In the main query we select the province_name and the town_name and then calculate the percentage of people using each source type, using the
CASE statements.
Then we join town_totals to combined_analysis_table, but this time the town_names are not unique, so we have to join town_totals, but we
check that both the province_name and town_name matches the values in combined_analysis_table.

Then we group it by province_name, then town_name. This query can take a while to calculate, so hopefully, you start to see how a query can
quickly become slow as it becomes more complex.

Before we jump into the data, let's store it as a temporary table first, so it is quicker to access.

Temporary tables in SQL are a nice way to store the results of a complex query. We run the query once, and the results are stored as a table. The
catch? If you close the database connection, it deletes the table, so you have to run it again each time you start working in MySQL. The benefit is
that we can use the table to do more calculations, without running the whole query each time.

In [39]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to create the temporary table
create_temp_table_query = '''
CREATE TEMPORARY TABLE town_aggregated_water_access AS
WITH town_totals AS  (
    SELECT province_name, town_name, SUM(people_served) AS total_ppl_serv
    FROM combined_analysis_table
    GROUP BY province_name, town_name
)
SELECT
    ct.province_name,
    ct.town_name,
    ROUND((SUM(CASE WHEN source_type = 'river'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS river,
    ROUND((SUM(CASE WHEN source_type = 'shared_tap'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS shared_tap,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home_broken'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home_broken,
    ROUND((SUM(CASE WHEN source_type = 'well'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS well
FROM
    combined_analysis_table ct
JOIN
    town_totals tt ON ct.province_name = tt.province_name AND ct.town_name = tt.town_name
GROUP BY
    ct.province_name,
    ct.town_name
ORDER BY
    ct.province_name;
'''

# Execute the query to create the temporary table
cursor = conn.cursor()
cursor.execute(create_temp_table_query)

# Now fetch the data from the temporary table
select_query = '''
SELECT * FROM town_aggregated_water_access;
'''

# Execute the select query and load the data into a pandas DataFrame
df = pd.read_sql_query(select_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


   province_name town_name  river  shared_tap  tap_in_home  \
0         Akatsi    Harare    2.0        17.0         28.0   
1         Akatsi  Kintampo    2.0        15.0         31.0   
2         Akatsi    Lusaka    2.0        17.0         28.0   
3         Akatsi     Rural    6.0        59.0          9.0   
4         Amanzi   Abidjan    2.0        53.0         22.0   
5         Amanzi     Amina    8.0        24.0          3.0   
6         Amanzi    Asmara    3.0        49.0         24.0   
7         Amanzi     Bello    3.0        53.0         20.0   
8         Amanzi    Dahabu    3.0        37.0         55.0   
9         Amanzi     Pwani    3.0        53.0         20.0   
10        Amanzi     Rural    3.0        27.0         30.0   
11       Hawassa     Amina    2.0        14.0         19.0   
12       Hawassa      Deka    3.0        16.0         23.0   
13       Hawassa    Djenne    3.0        18.0         19.0   
14       Hawassa     Rural    4.0        52.0         12.0   
15      

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/1866595890.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(select_query, conn)


So, let's order the results set by each column. If we order river DESC it confirms what we saw on a provincial level; People are drinking river water
in Sokoto.

In [41]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# SQL query to create the temporary table
create_temp_table_query = '''
CREATE TEMPORARY TABLE town_aggregated_water_access AS
WITH town_totals AS  (
    SELECT province_name, town_name, SUM(people_served) AS total_ppl_serv
    FROM combined_analysis_table
    GROUP BY province_name, town_name
)
SELECT
    ct.province_name,
    ct.town_name,
    ROUND((SUM(CASE WHEN source_type = 'river'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS river,
    ROUND((SUM(CASE WHEN source_type = 'shared_tap'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS shared_tap,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home_broken'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home_broken,
    ROUND((SUM(CASE WHEN source_type = 'well'
        THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS well
FROM
    combined_analysis_table ct
JOIN
    town_totals tt ON ct.province_name = tt.province_name AND ct.town_name = tt.town_name
GROUP BY
    ct.province_name,
    ct.town_name
ORDER BY
    river DESC,
    shared_tap DESC,
    tap_in_home DESC,
    tap_in_home_broken DESC,
    well DESC;
'''

# Execute the query to create the temporary table
cursor = conn.cursor()
cursor.execute(create_temp_table_query)

# Now fetch the data from the temporary table
select_query = '''
SELECT * FROM town_aggregated_water_access;
'''

# Execute the select query and load the data into a pandas DataFrame
df = pd.read_sql_query(select_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


   province_name town_name  river  shared_tap  tap_in_home  \
0         Sokoto     Rural   22.0        49.0          8.0   
1         Sokoto    Bahari   21.0        11.0         36.0   
2         Sokoto      Kofi   20.0        16.0         34.0   
3         Sokoto    Cheche   19.0        16.0         35.0   
4         Sokoto   Majengo   18.0        14.0         36.0   
5         Sokoto    Marang   17.0        19.0         31.0   
6         Sokoto    Ilanga   16.0        12.0         36.0   
7       Kilimani     Rural    9.0        55.0          8.0   
8       Kilimani      Zuri    8.0        71.0          6.0   
9         Amanzi     Amina    8.0        24.0          3.0   
10      Kilimani     Amara    8.0        22.0         25.0   
11      Kilimani   Isiqalo    7.0        19.0         25.0   
12      Kilimani    Mrembo    7.0        16.0         25.0   
13      Kilimani    Harare    7.0        11.0         30.0   
14        Akatsi     Rural    6.0        59.0          9.0   
15      

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3241200018.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(select_query, conn)


we look at the tap_in_home percentages in Sokoto too. Some of the citizens are forced to drink unsafe water from a river, while a lot of people
have running water in their homes in Sokoto. Large disparities in water access like this often show that the wealth distribution in Sokoto is very un-
equal. We should mention this in our report. We should also send our drilling teams to Sokoto first to drill some wells for the people who are drink-
ing river water, specifically the rural parts and the city of Bahari.

In [42]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Mayar123@',
    database='md_water_services'
)

# SQL query
query = '''
WITH town_totals AS (
    SELECT
        province_name,
        town_name,
        SUM(people_served) AS total_ppl_serv
    FROM
        combined_analysis_table
    GROUP BY
        province_name, town_name
)
SELECT
    ct.province_name,
    ct.town_name,
    ROUND((SUM(CASE WHEN source_type = 'river'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS river,
    ROUND((SUM(CASE WHEN source_type = 'shared_tap'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS shared_tap,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home_broken'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home_broken,
    ROUND((SUM(CASE WHEN source_type = 'well'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS well
FROM
    combined_analysis_table ct
JOIN
    town_totals tt ON ct.province_name = tt.province_name AND ct.town_name = tt.town_name
GROUP BY
    ct.province_name, ct.town_name
ORDER BY
    river DESC, ct.province_name;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)

# Filter the data for Amina in Amanzi
amina_data = df[df['town_name'] == 'Amina']
print(amina_data)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/4073645885.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


   province_name town_name  river  shared_tap  tap_in_home  \
0         Sokoto     Rural   22.0        49.0          8.0   
1         Sokoto    Bahari   21.0        11.0         36.0   
2         Sokoto      Kofi   20.0        16.0         34.0   
3         Sokoto    Cheche   19.0        16.0         35.0   
4         Sokoto   Majengo   18.0        14.0         36.0   
5         Sokoto    Marang   17.0        19.0         31.0   
6         Sokoto    Ilanga   16.0        12.0         36.0   
7       Kilimani     Rural    9.0        55.0          8.0   
8         Amanzi     Amina    8.0        24.0          3.0   
9       Kilimani      Zuri    8.0        71.0          6.0   
10      Kilimani     Amara    8.0        22.0         25.0   
11      Kilimani    Harare    7.0        11.0         30.0   
12      Kilimani   Isiqalo    7.0        19.0         25.0   
13      Kilimani    Mrembo    7.0        16.0         25.0   
14        Akatsi     Rural    6.0        59.0          9.0   
15      

Here only 3% of Amina's citizens have access to running tap
water in their homes. More than half of the people in Amina have taps installed in their homes, but they are not working. We should send out teams
to go and fix the infrastructure in Amina first. Fixing taps in people's homes, means those people don't have to queue for water anymore, so the
queues in Amina will also get shorter!

In [45]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Mayar123@',
    database='md_water_services'
)

# SQL query to create the temporary table
create_temp_table_query = '''
CREATE TEMPORARY TABLE town_aggregated_water_access AS
WITH town_totals AS (
    SELECT
        province_name,
        town_name,
        SUM(people_served) AS total_ppl_serv
    FROM
        combined_analysis_table
    GROUP BY
        province_name, town_name
)
SELECT
    ct.province_name,
    ct.town_name,
    ROUND((SUM(CASE WHEN source_type = 'river'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS river,
    ROUND((SUM(CASE WHEN source_type = 'shared_tap'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS shared_tap,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home,
    ROUND((SUM(CASE WHEN source_type = 'tap_in_home_broken'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS tap_in_home_broken,
    ROUND((SUM(CASE WHEN source_type = 'well'
            THEN people_served ELSE 0 END) * 100.0 / tt.total_ppl_serv), 0) AS well
FROM
    combined_analysis_table ct
JOIN
    town_totals tt ON ct.province_name = tt.province_name AND ct.town_name = tt.town_name
GROUP BY
    ct.province_name, ct.town_name
ORDER BY
    river DESC, ct.province_name;
'''

# Execute the query to create the temporary table
cursor = conn.cursor()
cursor.execute(create_temp_table_query)

# SQL query to find the town with the highest ratio of broken taps
find_broken_taps_query = '''
SELECT
    province_name,
    town_name,
    ROUND(tap_in_home_broken / (tap_in_home_broken + tap_in_home) * 100, 0) AS Pct_broken_taps
FROM
    town_aggregated_water_access
ORDER BY
    Pct_broken_taps DESC
LIMIT 100;
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(find_broken_taps_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


   province_name town_name  Pct_broken_taps
0         Amanzi     Amina             95.0
1       Kilimani      Zuri             65.0
2        Hawassa     Amina             56.0
3        Hawassa    Djenne             55.0
4       Kilimani     Rural             53.0
5         Amanzi     Bello             52.0
6        Hawassa   Yaounde             51.0
7         Amanzi     Pwani             51.0
8        Hawassa    Serowe             50.0
9        Hawassa     Rural             50.0
10        Amanzi     Rural             50.0
11        Akatsi    Lusaka             50.0
12        Sokoto     Rural             50.0
13        Akatsi    Harare             49.0
14       Hawassa      Deka             48.0
15      Kilimani    Mrembo             46.0
16        Akatsi  Kintampo             46.0
17        Amanzi   Abidjan             46.0
18        Amanzi    Asmara             45.0
19       Hawassa  Zanzibar             44.0
20      Kilimani   Isiqalo             42.0
21      Kilimani    Harare      

/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3174420447.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(find_broken_taps_query, conn)


We can see that Amina has infrastructure installed, but almost none of it is working, and only the capital city, Dahabu's water infrastructure works.
Strangely enough, all of the politicians of the past government lived in Dahabu, so they made sure they had water.

# Summary report

Insights

Ok, so let's sum up the data we have.
A couple of weeks ago we found some interesting insights:
1. Most water sources are rural in Maji Ndogo.
2. 43% of our people are using shared taps. 2000 people often share one tap.
3. 31% of our population has water infrastructure in their homes, but within that group,
4. 45% face non-functional systems due to issues with pipes, pumps, and reservoirs. Towns like Amina, the rural parts of Amanzi, and a couple
of towns across Akatsi and Hawassa have broken infrastructure.
5. 18% of our people are using wells of which, but within that, only 28% are clean. These are mostly in Hawassa, Kilimani and Akatsi.
6. Our citizens often face long wait times for water, averaging more than 120 minutes:
• Queues are very long on Saturdays.
• Queues are longer in the mornings and evenings.
• Wednesdays and Sundays have the shortest queues.

Plan of action

1. We want to focus our efforts on improving the water sources that affect the most people.
• Most people will benefit if we improve the shared taps first.
2. Wells are a good source of water, but many are contaminated. Fixing this will benefit a lot of people.
3. Fixing existing infrastructure will help many people. If they have running water again, they won't have to queue, thereby shorting queue times
for others. So we can solve two problems at once.
4. Installing taps in homes will stretch our resources too thin, so for now if the queue times are low, we won't improve that source.
5. Most water sources are in rural areas. We need to ensure our teams know this as this means they will have to make these repairs/upgrades in
rural areas where road conditions, supplies, and labour are harder challenges to overcome.

Practical solutions:

1. If communities are using rivers, we will dispatch trucks to those regions to provide water temporarily in the short term, while we send out
crews to drill for wells, providing a more permanent solution. Sokoto is the first province we will target.
2. If communities are using wells, we will install filters to purify the water. For chemically polluted wells, we can install reverse osmosis (RO)
filters, and for wells with biological contamination, we can install UV filters that kill microorganisms - but we should install RO filters too. In
the long term, we must figure out why these sources are polluted.
3. For shared taps, in the short term, we can send additional water tankers to the busiest taps, on the busiest days. We can use the queue time
pivot table we made to send tankers at the busiest times. Meanwhile, we can start the work on installing extra taps where they are needed.
According to UN standards, the maximum acceptable wait time for water is 30 minutes. With this in mind, our aim is to install taps to get
queue times below 30 min. Towns like Bello, Abidjan and Zuri have a lot of people using shared taps, so we will send out teams to those
towns first.
4. Shared taps with short queue times (< 30 min) represent a logistical challenge to further reduce waiting times. The most effective solution,
installing taps in homes, is resource-intensive and better suited as a long-term goal.
5. Addressing broken infrastructure offers a significant impact even with just a single intervention. It is expensive to fix, but so many people can
benefit from repairing one facility. For example, fixing a reservoir or pipe that multiple taps are connected to. We identified towns like Amina,
Lusaka, Zuri, Djenne and rural parts of Amanzi seem to be good places to start.

### A practical plan

Our final goal is to implement our plan in the database.
We have a plan to improve the water access in Maji Ndogo, so we need to think it through, and as our final task, create a table where our teams
have the information they need to fix, upgrade and repair water sources. They will need the addresses of the places they should visit (street
address, town, province), the type of water source they should improve, and what should be done to improve it.
We should also make space for them in the database to update us on their progress. We need to know if the repair is complete, and the date it was
completed, and give them space to upgrade the sources. Let's call this table Project_progress.

In [47]:

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to create the Project_progress table
create_table_query = '''
CREATE TABLE Project_progress (
    Project_id SERIAL PRIMARY KEY,
    source_id VARCHAR(20) NOT NULL REFERENCES water_source(source_id) 
        ON DELETE CASCADE 
        ON UPDATE CASCADE,
    Address VARCHAR(50),
    Town VARCHAR(30),
    Province VARCHAR(30),
    Source_type VARCHAR(50),
    Improvement VARCHAR(50),
    Source_status VARCHAR(50) DEFAULT 'Backlog' 
        CHECK (Source_status IN ('Backlog', 'In progress', 'Complete')),
    Date_of_completion DATE,
    Comments TEXT
);
'''

# Execute the query to create the table
try:
    cursor.execute(create_table_query)
    print("Table `Project_progress` created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Close the cursor and the connection
cursor.close()
conn.close()


Table `Project_progress` created successfully.


At a high level, the Improvements are as follows:
1. Rivers → Drill wells
2. wells: if the well is contaminated with chemicals → Install RO filter
3. wells: if the well is contaminated with biological contaminants → Install UV and RO filter
4. shared_taps: if the queue is longer than 30 min (30 min and above) → Install X taps nearby where X number of taps is calculated using X
= FLOOR(time_in_queue / 30).
5. tap_in_home_broken → Diagnose local infrastructure

Can you see that for wells and shared taps we have some IF logic, so we should be thinking CASE functions! Let's take the various Improvements
one by one, then combine them into one query at the end.

In [48]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to retrieve data from the relevant tables
project_progress_query = '''
SELECT
    location.address,
    location.town_name,
    location.province_name,
    water_source.source_id,
    water_source.type_of_water_source,
    well_pollution.results
FROM
    water_source
LEFT JOIN
    well_pollution ON water_source.source_id = well_pollution.source_id
INNER JOIN
    visits ON water_source.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(project_progress_query, conn)

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/1040835047.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(project_progress_query, conn)


                    address town_name province_name     source_id  \
0        2 Addis Ababa Road    Harare        Akatsi  AkHa00000224   
1       10 Addis Ababa Road    Harare        Akatsi  AkHa00001224   
2        9 Addis Ababa Road    Harare        Akatsi  AkHa00002224   
3      139 Addis Ababa Road    Harare        Akatsi  AkHa00003224   
4       17 Addis Ababa Road    Harare        Akatsi  AkHa00004224   
...                     ...       ...           ...           ...   
60141  183 Angelique Street     Rural        Sokoto  SoRu39645224   
60142   89 Angelique Street     Rural        Sokoto  SoRu39646224   
60143  116 Angelique Street     Rural        Sokoto  SoRu39647224   
60144    9 Angelique Street     Rural        Sokoto  SoRu39648224   
60145  150 Angelique Street     Rural        Sokoto  SoRu39649224   

      type_of_water_source                   results  
0              tap_in_home                      None  
1       tap_in_home_broken                      None  
2     

It joins the location, visits, and well_pollution tables to the water_source table. Since well_pollution only has data for wells, we have
to join those records to the water_source table with a LEFT JOIN and we used visits to link the various id's together.

First things first, let's filter the data to only contain sources we want to improve by thinking through the logic first.
1. Only records with visit_count = 1 are allowed.
2. Any of the following rows can be included:
a. Where shared taps have queue times over 30 min.
b. Only wells that are contaminated are allowed -- So we exclude wells that are Clean
c. Include any river and tap_in_home_broken sources.

In [64]:
import mysql.connector
import pandas as pd

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to retrieve filtered data from the relevant tables
project_progress_query = '''
SELECT
    location.address,
    location.town_name,
    location.province_name,
    water_source.source_id,
    water_source.type_of_water_source,
    well_pollution.results

FROM
    water_source
LEFT JOIN
    well_pollution ON water_source.source_id = well_pollution.source_id
INNER JOIN
    visits ON water_source.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
WHERE
    visits.visit_count = 1
    AND (
        well_pollution.results != 'Clean'
        OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
        OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue>=30)
    )

'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(project_progress_query, conn)

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/3906075700.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(project_progress_query, conn)


                             address town_name province_name     source_id  \
0            36 Pwani Mchangani Road    Ilanga        Sokoto  SoIl32582224   
1              129 Ziwa La Kioo Road     Rural      Kilimani  KiRu28935224   
2             18 Mlima Tazama Avenue     Rural       Hawassa  HaRu19752224   
3                 100 Mogadishu Road    Lusaka        Akatsi  AkLu01628224   
4           26 Bahari Ya Faraja Road     Rural      Kilimani  KiRu29315224   
...                              ...       ...           ...           ...   
25329        39 Mawimbi Mafupi Drive   Majengo        Sokoto  SoMa34164224   
25330  114 Mlima Wa Matumaini Street    Mrembo      Kilimani  KiMr24857224   
25331             110 N'Djamena Road     Rural      Kilimani  KiRu26812224   
25332           47 Desmond Tutu Road     Rural        Amanzi  AmRu14506224   
25333       129 African Skies Street     Rural        Akatsi  AkRu08520224   

      type_of_water_source                   results  
0       

Step 1: Wells
Let's start with wells. Depending on whether they are chemically contaminated, or biologically contaminated — we'll decide on the interventions.

In [65]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to retrieve filtered data from the relevant tables
project_progress_query = '''
SELECT
    location.address,
    location.town_name,
    location.province_name,
    water_source.source_id,
    water_source.type_of_water_source,
    well_pollution.results,
    CASE
        -- If the well is contaminated with biological contaminants
        WHEN well_pollution.results = 'Contaminated: Biological' THEN 'Install UV and RO filter'
        -- If the well is contaminated with chemical contaminants
        WHEN well_pollution.results = 'Contaminated: Chemical' THEN 'Install RO filter'
        -- For all other cases, set the improvement to NULL
        ELSE NULL
    END AS Improvement
FROM
    water_source
LEFT JOIN
    well_pollution ON water_source.source_id = well_pollution.source_id
INNER JOIN
    visits ON water_source.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
WHERE
    visits.visit_count = 1
    AND (
        well_pollution.results != 'Clean'
        OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
        OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue>=30)
    )


'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(project_progress_query, conn)

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/2870702026.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(project_progress_query, conn)


                             address town_name province_name     source_id  \
0            36 Pwani Mchangani Road    Ilanga        Sokoto  SoIl32582224   
1              129 Ziwa La Kioo Road     Rural      Kilimani  KiRu28935224   
2             18 Mlima Tazama Avenue     Rural       Hawassa  HaRu19752224   
3                 100 Mogadishu Road    Lusaka        Akatsi  AkLu01628224   
4           26 Bahari Ya Faraja Road     Rural      Kilimani  KiRu29315224   
...                              ...       ...           ...           ...   
25329        39 Mawimbi Mafupi Drive   Majengo        Sokoto  SoMa34164224   
25330  114 Mlima Wa Matumaini Street    Mrembo      Kilimani  KiMr24857224   
25331             110 N'Djamena Road     Rural      Kilimani  KiRu26812224   
25332           47 Desmond Tutu Road     Rural        Amanzi  AmRu14506224   
25333       129 African Skies Street     Rural        Akatsi  AkRu08520224   

      type_of_water_source                   results           

Step 2: Rivers
Now for the rivers. We upgrade those by drilling new wells nearby.

In [66]:


# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost'
    user='root',             # e.g., 'root'
    password='Mayar123@',    # e.g., 'password'
    database='md_water_services' # e.g., 'your_db'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to retrieve filtered data from the relevant tables
project_progress_query = '''
SELECT
    location.address,
    location.town_name,
    location.province_name,
    water_source.source_id,
    water_source.type_of_water_source,
    well_pollution.results,
    CASE
         -- For river sources, add 'Drill well' to Improvements
                    WHEN water_source.type_of_water_source = 'river' THEN 'Drill well'
                    -- If the well is contaminated with biological contaminants
                    WHEN well_pollution.results = 'Contaminated: Biological' THEN 'Install UV and RO filter'
                    -- If the well is contaminated with chemical contaminants
                    WHEN well_pollution.results = 'Contaminated: Chemical' THEN 'Install RO filter'
                    -- For other cases, including wells that are clean, the improvement is NULL
                    ELSE NULL
    END AS Improvement
FROM
    water_source
LEFT JOIN
    well_pollution ON water_source.source_id = well_pollution.source_id
INNER JOIN
    visits ON water_source.source_id = visits.source_id
INNER JOIN
    location ON location.location_id = visits.location_id
WHERE
    visits.visit_count = 1
    AND (
        well_pollution.results != 'Clean'
        OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
        OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue>=30)
    )

'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(project_progress_query, conn)

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


/var/folders/_p/z0wlrjjj3n7d6kwjc9cm54sm0000gn/T/ipykernel_6262/4094389167.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(project_progress_query, conn)


                             address town_name province_name     source_id  \
0            36 Pwani Mchangani Road    Ilanga        Sokoto  SoIl32582224   
1              129 Ziwa La Kioo Road     Rural      Kilimani  KiRu28935224   
2             18 Mlima Tazama Avenue     Rural       Hawassa  HaRu19752224   
3                 100 Mogadishu Road    Lusaka        Akatsi  AkLu01628224   
4           26 Bahari Ya Faraja Road     Rural      Kilimani  KiRu29315224   
...                              ...       ...           ...           ...   
25329        39 Mawimbi Mafupi Drive   Majengo        Sokoto  SoMa34164224   
25330  114 Mlima Wa Matumaini Street    Mrembo      Kilimani  KiMr24857224   
25331             110 N'Djamena Road     Rural      Kilimani  KiRu26812224   
25332           47 Desmond Tutu Road     Rural        Amanzi  AmRu14506224   
25333       129 African Skies Street     Rural        Akatsi  AkRu08520224   

      type_of_water_source                   results           

Step 3: Shared taps
Next up, shared taps. We need to install one tap near each shared tap for every 30 min of queue time.

In [70]:


# Create a connection to the database
try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Mayar123@',
        database='md_water_services'
    )
    
    if conn.is_connected():
        print("Successfully connected to the database")
        
        # Define the SQL query
        query = '''
        SELECT
            location.address,
            location.town_name,
            location.province_name,
            water_source.source_id,
            water_source.type_of_water_source,
            well_pollution.results,
            visits.time_in_queue,
            CASE
                -- For river sources, add 'Drill well' to Improvements
                WHEN water_source.type_of_water_source = 'river' THEN 'Drill well'
                -- If the well is contaminated with biological contaminants
                WHEN well_pollution.results = 'Contaminated: Biological' THEN 'Install UV and RO filter'
                -- If the well is contaminated with chemical contaminants
                WHEN well_pollution.results = 'Contaminated: Chemical' THEN 'Install RO filter'
                -- For shared taps, calculate the number of additional taps based on queue time
                WHEN water_source.type_of_water_source = 'shared_tap'
                     AND visits.time_in_queue >= 30 THEN CONCAT('Install ', FLOOR(visits.time_in_queue / 30), ' taps nearby')
                -- For other cases, including wells that are clean, the improvement is NULL
                ELSE NULL
            END AS Improvement
        FROM
            water_source
        LEFT JOIN
            well_pollution ON water_source.source_id = well_pollution.source_id
        INNER JOIN
            visits ON water_source.source_id = visits.source_id
        INNER JOIN
            location ON location.location_id = visits.location_id
       WHERE
    visits.visit_count = 1
    AND (
        well_pollution.results != 'Clean'
        OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
        OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue>=30)
    )
        '''

        # Execute the query
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch the data into a pandas DataFrame
        df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        # Display the DataFrame
        print(df)

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()


Successfully connected to the database
                             address town_name province_name     source_id  \
0            36 Pwani Mchangani Road    Ilanga        Sokoto  SoIl32582224   
1              129 Ziwa La Kioo Road     Rural      Kilimani  KiRu28935224   
2             18 Mlima Tazama Avenue     Rural       Hawassa  HaRu19752224   
3                 100 Mogadishu Road    Lusaka        Akatsi  AkLu01628224   
4           26 Bahari Ya Faraja Road     Rural      Kilimani  KiRu29315224   
...                              ...       ...           ...           ...   
25329        39 Mawimbi Mafupi Drive   Majengo        Sokoto  SoMa34164224   
25330  114 Mlima Wa Matumaini Street    Mrembo      Kilimani  KiMr24857224   
25331             110 N'Djamena Road     Rural      Kilimani  KiRu26812224   
25332           47 Desmond Tutu Road     Rural        Amanzi  AmRu14506224   
25333       129 African Skies Street     Rural        Akatsi  AkRu08520224   

      type_of_water_sour

Step 4: In-home taps

Lastly, let's look at in-home taps, specifically broken ones. These taps indicate broken infrastructure. So these need to be inspected by our engi-
neers.

In [71]:


# Create a connection to the database
try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Mayar123@',
        database='md_water_services'
    )
    
    if conn.is_connected():
        print("Successfully connected to the database")
        
        # Define the SQL query
        query = '''
        SELECT
            location.address,
            location.town_name,
            location.province_name,
            water_source.source_id,
            water_source.type_of_water_source,
            well_pollution.results,
            visits.time_in_queue,
            CASE
                -- For river sources, add 'Drill well' to Improvements
                WHEN water_source.type_of_water_source = 'river' THEN 'Drill well'
                -- If the well is contaminated with biological contaminants
                WHEN well_pollution.results = 'Contaminated: Biological' THEN 'Install UV and RO filter'
                -- If the well is contaminated with chemical contaminants
                WHEN well_pollution.results = 'Contaminated: Chemical' THEN 'Install RO filter'
                -- For shared taps, calculate the number of additional taps based on queue time
                WHEN water_source.type_of_water_source = 'shared_tap'
                     AND visits.time_in_queue >= 30 THEN CONCAT('Install ', FLOOR(visits.time_in_queue / 30), ' taps nearby')
                -- For in-home broken taps, diagnose local infrastructure
                WHEN water_source.type_of_water_source = 'tap_in_home_broken' THEN 'Diagnose local infrastructure'
                -- For other cases, including wells that are clean, the improvement is NULL
                ELSE NULL
            END AS Improvement
        FROM
            water_source
        LEFT JOIN
            well_pollution ON water_source.source_id = well_pollution.source_id
        INNER JOIN
            visits ON water_source.source_id = visits.source_id
        INNER JOIN
            location ON location.location_id = visits.location_id
       WHERE
    visits.visit_count = 1
    AND (
        well_pollution.results != 'Clean'
        OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
        OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue>=30)
    )
        '''

        # Execute the query
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch the data into a pandas DataFrame
        df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        # Display the DataFrame
        print(df)

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()


Successfully connected to the database
                             address town_name province_name     source_id  \
0            36 Pwani Mchangani Road    Ilanga        Sokoto  SoIl32582224   
1              129 Ziwa La Kioo Road     Rural      Kilimani  KiRu28935224   
2             18 Mlima Tazama Avenue     Rural       Hawassa  HaRu19752224   
3                 100 Mogadishu Road    Lusaka        Akatsi  AkLu01628224   
4           26 Bahari Ya Faraja Road     Rural      Kilimani  KiRu29315224   
...                              ...       ...           ...           ...   
25329        39 Mawimbi Mafupi Drive   Majengo        Sokoto  SoMa34164224   
25330  114 Mlima Wa Matumaini Street    Mrembo      Kilimani  KiMr24857224   
25331             110 N'Djamena Road     Rural      Kilimani  KiRu26812224   
25332           47 Desmond Tutu Road     Rural        Amanzi  AmRu14506224   
25333       129 African Skies Street     Rural        Akatsi  AkRu08520224   

      type_of_water_sour

Step 5: Add the data to Project_progress

In [73]:
import mysql.connector
import pandas as pd

# Create a connection to the database
try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Mayar123@',
        database='md_water_services'
    )
    
    if conn.is_connected():
        print("Successfully connected to the database")

        # Define the SQL query
        query = '''
        SELECT
            location.address,
            location.town_name,
            location.province_name,
            water_source.source_id,
            water_source.type_of_water_source,
            well_pollution.results,
            visits.time_in_queue,
            CASE
                -- For river sources, add 'Drill well' to Improvements
                WHEN water_source.type_of_water_source = 'river' THEN 'Drill well'
                -- If the well is contaminated with biological contaminants
                WHEN well_pollution.results = 'Contaminated: Biological' THEN 'Install UV and RO filter'
                -- If the well is contaminated with chemical contaminants
                WHEN well_pollution.results = 'Contaminated: Chemical' THEN 'Install RO filter'
                -- For shared taps, calculate the number of additional taps based on queue time
                WHEN water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue >= 30
                THEN CONCAT('Install ', FLOOR(visits.time_in_queue / 30), ' taps nearby')
                -- For in-home broken taps, diagnose local infrastructure
                WHEN water_source.type_of_water_source = 'tap_in_home_broken' THEN 'Diagnose local infrastructure'
                -- For other cases, including wells that are clean, the improvement is NULL
                ELSE NULL
            END AS Improvement
        FROM
            water_source
        LEFT JOIN
            well_pollution ON water_source.source_id = well_pollution.source_id
        INNER JOIN
            visits ON water_source.source_id = visits.source_id
        INNER JOIN
            location ON location.location_id = visits.location_id
        WHERE
            visits.visit_count = 1
            AND (
                well_pollution.results != 'Clean'
                OR water_source.type_of_water_source IN ('tap_in_home_broken', 'river')
                OR (water_source.type_of_water_source = 'shared_tap' AND visits.time_in_queue >= 30)
            )
        '''

        # Execute the query and fetch the data
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        # Print fetched data for debugging
        print("Fetched data:")
        for row in rows[:5]:  # Print the first 5 rows
            print(row)
        
        # Prepare the data for insertion into Project_progress
        insert_query = '''
        INSERT INTO Project_progress (
            source_id,
            Address,
            Town,
            Province,
            Source_type,
            Improvement
        ) VALUES (%s, %s, %s, %s, %s, %s)
        '''

        # Ensure the data fetched matches the number of placeholders
        data_to_insert = [tuple(row[3:]) + (row[5],) for row in rows]  # Extract the relevant data

        # Insert data into Project_progress table
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        
        print(f"Inserted {cursor.rowcount} rows into Project_progress")

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("Connection closed")


Successfully connected to the database
Fetched data:
('36 Pwani Mchangani Road', 'Ilanga', 'Sokoto', 'SoIl32582224', 'river', None, 15, 'Drill well')
('129 Ziwa La Kioo Road', 'Rural', 'Kilimani', 'KiRu28935224', 'well', 'Contaminated: Biological', 0, 'Install UV and RO filter')
('18 Mlima Tazama Avenue', 'Rural', 'Hawassa', 'HaRu19752224', 'shared_tap', None, 62, 'Install 2 taps nearby')
('100 Mogadishu Road', 'Lusaka', 'Akatsi', 'AkLu01628224', 'well', 'Contaminated: Biological', 0, 'Install UV and RO filter')
('26 Bahari Ya Faraja Road', 'Rural', 'Kilimani', 'KiRu29315224', 'river', None, 9, 'Drill well')
Inserted 25334 rows into Project_progress
Connection closed


There we go, all done! Now we send off our summary report to Pres. Naledi with our main findings, so they can start organising the teams. We'll
also explain the Project_progress table, and how this will help us track our progress.